In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = "TRUE"

In [3]:
b_size = 5

train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True,     #수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,  #전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1, #마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,
                                  #shear_range=0.7,
                                  #zoom_range=[0.9, 2.2],
                                  #vertical_flip=True,
                                  fill_mode='nearest') #주위의 px 값으로 채움

In [4]:
train_generator = train_datagen.flow_from_directory(
       'data/train',   #학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=b_size,
       class_mode='binary')

#테스트 셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(
       'data/test',   #테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=b_size,
       class_mode='binary')

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [5]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False
transfer_model.summary()

58889256/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [7]:
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(1, activation='sigmoid'))
finetune_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 64)                524352    
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 15,239,105
Trainable params: 524,417
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
finetune_model.compile(loss='binary_crossentropy', 
                       optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

In [9]:
history=finetune_model.fit(train_generator,
                  epochs=20,
                  validation_data=test_generator)

Epoch 1/20
32/32 [==============================] - 18s 527ms/step - loss: 0.6867 - accuracy: 0.5500 - val_loss: 0.4835 - val_accuracy: 0.8167
Epoch 2/20
32/32 [==============================] - 16s 494ms/step - loss: 0.4066 - accuracy: 0.8875 - val_loss: 0.3678 - val_accuracy: 0.9167
Epoch 3/20
32/32 [==============================] - 16s 504ms/step - loss: 0.3296 - accuracy: 0.8938 - val_loss: 0.2921 - val_accuracy: 0.9417
Epoch 4/20
32/32 [==============================] - 17s 531ms/step - loss: 0.2344 - accuracy: 0.9563 - val_loss: 0.2469 - val_accuracy: 0.9500
Epoch 5/20
32/32 [==============================] - 16s 492ms/step - loss: 0.2006 - accuracy: 0.9750 - val_loss: 0.2204 - val_accuracy: 0.9583
Epoch 6/20
32/32 [==============================] - 15s 474ms/step - loss: 0.1923 - accuracy: 0.9500 - val_loss: 0.2248 - val_accuracy: 0.9167
Epoch 7/20
32/32 [==============================] - 15s 475ms/step - loss: 0.1653 - accuracy: 0.9500 - val_loss: 0.2428 - val_accuracy: 0.8917

#개, 고양이 vgg16으로 -- 학습시간은 줄어들고 정확도는 올라감